### Basic Problem using Pygmo

In [2]:
!pip install pygmo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.5 MB 5.2 MB/s 


In [23]:
import pygmo as pg
import time

In [24]:
#Try a knapsack problem
p = [10, 13, 18, 31, 7, 15]
w = [11, 15, 20, 35, 10, 33]
c, I = 47, range(len(w))

In [25]:
class udf:
    def fitness(self, x):
      obj = 0
      ci1 = 0
      obj = - (x[0]*p[0]+x[1]*p[1]+x[2]*p[2]+x[3]*p[3]+x[4]*p[4]+x[5]*p[5])
      ci1 = x[0]*w[0]+x[1]*w[1]+x[2]*w[2]+x[3]*w[3]+x[4]*w[4]+x[5]*w[5] - c
      return [obj,ci1]
    def get_nic(self):
      return 1
    def get_bounds(self):
      return ([0,0,0,0,0,0],[1,1,1,1,1,1])
    def gradient(self, x):
      return pg.estimate_gradient_h(lambda x: self.fitness(x), x)

In [26]:
prob = pg.problem(udf())
print(prob) 

Problem name: <class '__main__.udf'>
	C++ class name: pybind11::object

	Global dimension:			6
	Integer dimension:			0
	Fitness dimension:			2
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	1
	Tolerances on constraints: [0]
	Lower bounds: [0, 0, 0, 0, 0, ... ]
	Upper bounds: [1, 1, 1, 1, 1, ... ]
	Has batch fitness evaluation: false

	Has gradient: true
	User implemented gradient sparsity: false
	Expected gradients: 12
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 0
	Gradient evaluations: 0

	Thread safety: none



In [27]:
start = time.time()

In [28]:
algo = pg.algorithm(uda = pg.nlopt('auglag'))
algo.extract(pg.nlopt).local_optimizer = pg.nlopt('var2')
algo.set_verbosity(200) # in this case this correspond to logs each 200 objevals
pop = pg.population(prob = udf(), size = 1)
pop.problem.c_tol = [1E-6] * 1
pop = algo.evolve(pop) 

In [29]:
end = time.time()

In [30]:
print("Time taken = ",end-start)

Time taken =  0.1146700382232666


In [31]:
pop

Problem name: <class '__main__.udf'>
	C++ class name: pybind11::object

	Global dimension:			6
	Integer dimension:			0
	Fitness dimension:			2
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	1
	Tolerances on constraints: [1e-06]
	Lower bounds: [0, 0, 0, 0, 0, ... ]
	Upper bounds: [1, 1, 1, 1, 1, ... ]
	Has batch fitness evaluation: false

	Has gradient: true
	User implemented gradient sparsity: false
	Expected gradients: 12
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 350
	Gradient evaluations: 332

	Thread safety: none

Population size: 1

List of individuals: 
#0:
	ID:			938434405110322250
	Decision vector:	[1, 0, 1, 0.457143, 0, ... ]
	Fitness vector:		[-42.1714, 0]

Champion decision vector: [1, 0, 1, 0.457143, 0, ... ]
Champion fitness: [-42.1714, 0]

In [32]:
pop.champion_x 

array([1.        , 0.        , 1.        , 0.45714286, 0.        ,
       0.        ])

> This is solving the problem like Non Integer Programming

### Island approach

In [39]:
start2 = time.time()

In [40]:
isl = pg.island(algo = pg.de(10), prob = udf(), size=20, udi=pg.thread_island())

In [41]:
isl.evolve()

In [42]:
end2 = time.time()

In [43]:
print("Time taken by Island",end2-start2)

Time taken by Island 0.027341365814208984


In [44]:
isl.get_population().champion_f

array([-33.02342255,  -6.32984212])

In [45]:
isl

Island name: Thread island
	C++ class name: pagmo::thread_island

	Status: idle - **error occurred**

Extra info:
	Using pool: yes

Algorithm: DE: Differential Evolution

Problem: <class '__main__.udf'>

Replacement policy: Fair replace

Selection policy: Select best

Population size: 20
	Champion decision vector: [0.344263, 0.551594, 0.385044, 0.283539, 0.691195, ... ]
	Champion fitness: [-33.0234, -6.32984]

In [46]:
isl.get_population().champion_x 

array([0.3442629 , 0.55159412, 0.38504434, 0.28353851, 0.69119488,
       0.12341427])

> We can see that time taken by Island method is roughly 10 times faster compared to single thread approach